<br>
<br>
<br>
<br>
<br>
<br>
<br>
<center>
    <h1 style = "font-size:48px; font-weight:normal"><b>AI</b>-TECH</style></h1>
    <br>
<br>
<br>
<br>
    <h1 style = "font-size:24px; font-weight:normal">LABORATORIUM</style></h1>
    <h1 style = "font-size:24px">Przetwarzanie Multimediów w
Systemach Decyzyjnych
</style></h1>
    <h1 style = "font-size:24px">Segmentacja Semantyczna</style></h1>
    <h1 style = "font-size:24px">Przemysław Rośleń</style></h1>
    
   
</center>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
    </body>
    </html>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# 1. Konfiguracja środowiska

In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1
#INSTALL OLDER TORCH VERSION

#IMPORTANT
#After running this code block click RUNTIME -> RESTART SESSION
#and continue from the NEXT code block. This will allow to use older torch version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 887.3/887.5 MB 24.2 MB/s eta 0:00:01

In [ ]:
#!pip install mxnet==1.8.0
#!pip install gluoncv==0.10.4.post4
!pip install mxnet
!pip install gluoncv

import mxnet as mx

from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
ctx = mx.cpu(0)

#wizualizacja
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from gluoncv.utils.viz import get_color_pallete
import matplotlib.image as mpimg

# biblioteki pomocnicze
import time
import os
import cv2
import numpy as np

#1. Wczytanie wytrenowanego modelu na zbiorze Cityscapes

W celu wczytania wytrenowanego modelu użyjemy jeden z modeli dostępnych w **model zoo**:
 https://cv.gluon.ai/model_zoo/segmentation.html ze zbioru **Cityscapes**.

Ze względu na czas działania polecane są modele korzystające z **resnet50** oraz **fastscnn**.

**Uwaga:** model **DANET** ma niezgodny interfejs i jego użycie wymaga zmian w kodzie.
Model **deeplab_v3b_plus_wideresnet_citys** - ma problemy z działaniem w GoogleColab.

In [ ]:
model = gluoncv.model_zoo.get_model('fastscnn_citys', pretrained=True)

In [ ]:
# wczytanie danych z dysku google (dane dostarczone w załączniku do laboratorium)
# należy wgrać dane na dysk google
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/TODO/") #tutaj ustaw odpowiednią ściężkę

Uruchomienie modelu na przykładowym obrazie

In [ ]:
filepath = "imgs/val/munster/munster_000000_000019_leftImg8bit.png"

from gluoncv.data.transforms.presets.segmentation import test_transform

img = image.imread(filepath)
img = test_transform(img, ctx) # normalizacja obrazu
output = model.predict(img)
predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy() # argmax zwraca indeks przewidywanej klasy

In [ ]:
# wyświetlenie obrazu i maski
cv2_imshow(cv2.imread(filepath))
mask = get_color_pallete(predict, 'citys')
opencvmask = cv2.cvtColor(np.array(mask), cv2.COLOR_RGB2BGR)
plt.imshow(mask)
plt.show()

#Zadanie 1. Obliczenie metryki pixel accuracy [1 pkt.]

W tym ćwiczeniu Twoim zadaniem jest uzupełnienie kodu funkcji compute_pix_acc obliczącej proporcję poprawnie sklasyfikowanych pikseli.
**W funkcji należy ignorować piksele dla których etykieta (label) wynosi -1.**
Oczekiwana wartość wynosi około 90%.

In [ ]:
def compute_pix_acc(prediction, label):
    total_pixels = TODO
    correct_pixels = TODO
    return correct_pixels / total_pixels

In [ ]:
model = gluoncv.model_zoo.get_model('icnet_resnet50_citys', pretrained=True)

img_root_dir = "imgs/val/munster"
label_root_dir = "labels/val/munster"
NUM_CLASSES = 19
CLASSES = ("road", "sidewalk", "building", "wall", "fence",
               "pole", "traffic light", "traffic sign", "vegetation",
               "terrain", "sky", "person", "rider", "car", "truck",
               "bus", "train", "motorcycle", "bicyle")

# mapowanie z pliku label na klasy używane w zbiorze Cityscapes do ewaluacji
# wartość -1 oznacza klasy, które nie są brane pod uwagę przy ewaluacji ("tło", "niebo", "brak etykiety")
cityscapes_mapping = np.array([-1, -1, -1, -1, -1, -1,
                    -1, 0, 1, 2, -1, -1,
                    3, 4, 5, -1, -1, -1,
                    6, -1, 7, 8, 9, 10,
                    11, 12, 13, 14, 15, 16,
                    -1, -1, 17, 18])
mapper = lambda x: cityscapes_mapping[x]

imgs = os.listdir(img_root_dir)
filename = imgs[0]
label_name = filename.replace("leftImg8bit", "gtFine_labelIds")
label_path = os.path.join(label_root_dir, label_name)
img_path = os.path.join(img_root_dir, filename)
img = cv2.imread(img_path)
img = mx.nd.array(img)
img = test_transform(img, ctx)
output = model.predict(img)
predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()
predict = predict.astype('int')
label = cv2.imread(label_path).astype('int')[:,:,0]
label = mapper(label)
pix_acc = compute_pix_acc(predict, label)
print(pix_acc)


# Zadanie 2. Obliczenie metryki meanIoU [3 pkt.]
W tym celu należy dla każdej klasy obliczyć metrykę Intersection over Union i następnie obliczyć średnią dla każdej z klas. Przyjmij, że do ewaluacji bierzemy tylko klasy, które posiadają więcej niż 1500 pikseli w pliku etykiet (aby usunąć nieistotne obiekty)

In [ ]:
def compute_miou(prediction, label):
    class_accs= []
    num_classes = 19
    for class_id in range(0, num_classes):
        tp = TODO
        fp = TODO
        fn = TODO
        target_pix = np.sum(label == class_id)
        if (target_pix > 1500):
            iou = TODO
    return -1


img_root_dir = "imgs/val/munster"
label_root_dir = "labels/val/munster"
NUM_CLASS = 19

cityscapes_mapping = np.array([-1, -1, -1, -1, -1, -1,
                    -1, 0, 1, 2, -1, -1,
                    3, 4, 5, -1, -1, -1,
                    6, -1, 7, 8, 9, 10,
                    11, 12, 13, 14, 15, 16,
                    -1, -1, 17, 18])
mapper = lambda x: cityscapes_mapping[x]

imgs = os.listdir(img_root_dir)
filename = imgs[0]
label_name = filename.replace("leftImg8bit", "gtFine_labelIds")
label_path = os.path.join(label_root_dir, label_name)
img_path = os.path.join(img_root_dir, filename)
img = image.imread(img_path)
img = test_transform(img, ctx)
output = model.predict(img)
predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()
predict = predict.astype('int')
label = cv2.imread(label_path).astype('int')[:,:,0]
label = mapper(label)
pix_acc = compute_miou(predict, label)
print(pix_acc)

#Zadanie 3. Obliczenie dokładności dla 2 wybranych modeli [2 pkt.]

W tym zadaniu Twoim celem jest obliczenie dokładności pix_acc oraz mIoU dla 2 wybranych modeli. Zapisz również czas inferencji (Frames Per Second).

In [ ]:
model = gluoncv.model_zoo.get_model('icnet_resnet50_citys', pretrained=True)

NUM_TEST_IMGS = 20 #FOR FASTER DEBUGGIN

# ...
# ...

start = time.time()

for idx, filename in enumerate(imgs):
    label_name = filename.replace("leftImg8bit", "gtFine_labelIds")
    label_path = os.path.join(label_root_dir, label_name)
    img_path = os.path.join(img_root_dir, filename)
    img = cv2.imread(img_path)
    img = mx.nd.array(img)
    img = test_transform(img, ctx)
    print(idx)

    output = model.predict(img)

    predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

    predict = predict.astype('int')

    label = cv2.imread(label_path).astype('int')[:,:,0]
    label = mapper(label)

    #obliczenie metryki dla każdego obrazu

    if idx >= NUM_TEST_IMGS:
        break

# zapisz końcowe metryki na zbiorze danych

diff = time.time() - start
secs_per_image = diff / float(NUM_TEST_IMGS)

print("FPS", 1./secs_per_image)

# Zadanie 4. Obliczenie wrażliwości modeli na szum [1 pkt.]

Oblicz dokładność modeli po dodaniu szumu gaussowskiego (**dla dwóch różnych wartości wariancji**). W tym celu wykorzystaj funkcję napisaną w poprzednim ćwiczeniu laboratoryjnym.

Wyniki **dopisz do tabeli z poprzedniego zadania.**

In [ ]:
# Definicja funkcji dodającej szum gaussowski
def add_noise(input_img, var = 0.0):
    mean = 0.0
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,input_img.size)
    gauss = gauss.reshape(input_img.shape[0], input_img.shape[1], input_img.shape[2]).astype('uint8')
    # Dodaj szum do obrazu
    img_gauss = cv2.add(input_img,gauss)
    return img_gauss

In [ ]:
#TODO

# Zadanie 5. Model ensembling [3 pkt.]
W tym ćwiczeniu wykorzystmay wiele modeli (model ensemling) do poprawy dokładności działania modelu. **Użyj 2 wybranych przez Ciebie modeli** (zwróć uwagę na czas przetwarzania). W celu uzyskania końcowej predykcji można wziąć średnią arytmetyczną predykcji z modeli.
Przetestuj również wrażliwość tej techniki na szum (używając jednej z wariancji użytych w poprzednim ćwiczeniu).

W sprawozdaniu wklej kawałek kodu odpowiedzialny za agregację wyników.

In [ ]:
# TODO